<a href="https://colab.research.google.com/github/tw7366/Sentiment-Analysis-of-Movie-Reviews/blob/master/Sentiment%20Analysis%20of%20Movie%20Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import re

In [2]:
reviews_df=pd.read_csv('drive/My Drive/IMDB Dataset.csv')
reviews = np.array(reviews_df['review'])
sentiment = np.array(reviews_df['sentiment'])
print(reviews[:3])

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [3]:
def preprocess_review(review_list):
    """
    param: a list of string
    return: a list of string
    preprocess reviews by changing all alphabets to lowercase, removing punctuations, and cleaning html leftover code
    """
    for i, review in enumerate(review_list):
        review = review.replace('<br /><br />', ' ')
        review = review.lower()
        review = re.sub(r"[^A-Za-z0-9' ]+", '', review)
        review = review.replace('  ', ' ')
        review_list[i] = review
    return review_list

preprocessed_reviews = preprocess_review(reviews)
print(preprocessed_reviews[:3])

["one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare forget pretty 

In [4]:
# to divide train & test sets
test_sample_size = int(0.1*len(preprocessed_reviews))  # 10% of data as the validation set

# for sentiment
sentiment = [1 if x=='positive' else 0 for x in sentiment]

# separate data to train & test sets
X_test, X_train = (np.array(preprocessed_reviews[:test_sample_size]), 
                   np.array(preprocessed_reviews[test_sample_size:])
)

y_test, y_train = (np.array(sentiment[:test_sample_size]), 
                   np.array(sentiment[test_sample_size:])
)

tokenizer = Tokenizer(oov_token='<OOV>')  # for the unknown words
tokenizer.fit_on_texts(X_train)

vocab_count = len(tokenizer.word_index) + 1  # +1 is for padding

# create padded sequences
training_sequences = tokenizer.texts_to_sequences(X_train)  # tokenizer.word_index to see indexes
training_padded = pad_sequences(training_sequences, padding='post')  # pad sequences with 0s 

testing_sequences = tokenizer.texts_to_sequences(X_test)  # tokenizer.word_index to see indexes
testing_padded = pad_sequences(testing_sequences, padding='post')  # pad sequences with 0s 

input_length = len(testing_padded[0])  # length of all sequences


# build a model
model = keras.models.Sequential()
model.add(keras.layers.Embedding(input_dim=vocab_count,
                                 output_dim=6,
                                 input_length=input_length,
                                 mask_zero = True)
)
model.add(keras.layers.GlobalAveragePooling1D())  # find the average of vectors to get sentiment
model.add(keras.layers.Dense(128, activation='relu'))  # hidden layer
model.add(keras.layers.Dense(16, activation='relu'))  # hidden layer
model.add(keras.layers.Dense(1, activation='sigmoid'))  # output layer

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1707, 6)           1022112   
_________________________________________________________________
global_average_pooling1d (Gl (None, 6)                 0         
_________________________________________________________________
dense (Dense)                (None, 128)               896       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,025,089
Trainable params: 1,025,089
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.fit(training_padded, y_train, epochs=5, batch_size=512,
          validation_data=(testing_padded, y_test)
)

Epoch 1/5
88/88 [==============================] - 4s 48ms/step - loss: 0.6811 - accuracy: 0.6133 - val_loss: 0.6324 - val_accuracy: 0.7516
Epoch 2/5
88/88 [==============================] - 4s 47ms/step - loss: 0.4587 - accuracy: 0.8379 - val_loss: 0.3188 - val_accuracy: 0.8722
Epoch 3/5
88/88 [==============================] - 4s 48ms/step - loss: 0.2497 - accuracy: 0.9039 - val_loss: 0.2558 - val_accuracy: 0.8952
Epoch 4/5
88/88 [==============================] - 4s 47ms/step - loss: 0.1798 - accuracy: 0.9350 - val_loss: 0.2411 - val_accuracy: 0.9064
Epoch 5/5
88/88 [==============================] - 4s 47ms/step - loss: 0.1362 - accuracy: 0.9542 - val_loss: 0.2387 - val_accuracy: 0.9056


In [6]:
index_list = []
predictions = model.predict(testing_padded)
predictions = ['negative' if x <= 0.5 else 'positive' for x in predictions]
y_test = ['negative' if x <= 0.5 else 'positive' for x in y_test]
for i, (prediction, truth) in enumerate(zip(predictions, y_test)):
    if prediction != truth:
        index_list.append(i)

print(f'The model has predicted {len(index_list)} wrong of {len(predictions)}')

The model has predicted 472 wrong of 5000


In [7]:
for i, item in enumerate(index_list):
    if i <= 5:
        print(X_test[item])
        print(f'Prediction: {predictions[item]}')
        print(f'Sentiment: {y_test[item]}')
        print('============================')

the cast played shakespeare shakespeare lost i appreciate that this is trying to bring shakespeare to the masses but why ruin something so good is it because 'the scottish play' is my favorite shakespeare i do not know what i do know is that a certain rev bowdler hence bowdlerization tried to do something similar in the victorian era in other words you cannot improve perfection i have no more to write but as i have to write at least ten lines of text and english composition was never my forte i will just have to keep going and say that this movie as the saying goes just does not cut it
Prediction: positive
Sentiment: negative
some films just simply should not be remade this is one of them in and of itself it is not a bad film but it fails to capture the flavor and the terror of the 1963 film of the same title liam neeson was excellent as he always is and most of the cast holds up with the exception of owen wilson who just did not bring the right feel to the character of luke but the ma

The model made good assumptions for most of them, but it got easily confused by words like 'good', 'bad', 'well', etc. 

LSTM Model

In [8]:
y_test = [0 if x == 'negative' else 1 for x in y_test]

# the model takes numpy arrays as inputs
y_test = np.asarray(y_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

for i, (train_item, test_item) in enumerate(zip(training_padded, testing_padded)):
    training_padded[i] = np.asarray(train_item).astype(np.float32)
    testing_padded[i] = np.asarray(test_item).astype(np.float32)

In [9]:
from tensorflow.keras.layers import SpatialDropout1D, Embedding, LSTM, Bidirectional, Dense

new_model = keras.models.Sequential()
new_model.add(Embedding(input_dim=vocab_count,
                        output_dim=6,
                        input_length=input_length))
new_model.add(Bidirectional(tf.keras.layers.LSTM(16, dropout=0.2)))
new_model.add(Dense(16, activation='relu'))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

print(new_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1707, 6)           1022112   
_________________________________________________________________
bidirectional (Bidirectional (None, 32)                2944      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 1,025,601
Trainable params: 1,025,601
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
new_model.fit(training_padded, y_train, epochs=5, batch_size=512,
              validation_data=(testing_padded, y_test)
)

Epoch 1/5
88/88 [==============================] - 20s 226ms/step - loss: 0.6812 - accuracy: 0.5816 - val_loss: 0.6236 - val_accuracy: 0.7438
Epoch 2/5
88/88 [==============================] - 20s 223ms/step - loss: 0.4642 - accuracy: 0.8102 - val_loss: 0.3596 - val_accuracy: 0.8488
Epoch 3/5
88/88 [==============================] - 20s 222ms/step - loss: 0.2528 - accuracy: 0.9040 - val_loss: 0.2531 - val_accuracy: 0.9002
Epoch 4/5
88/88 [==============================] - 19s 221ms/step - loss: 0.1640 - accuracy: 0.9438 - val_loss: 0.2475 - val_accuracy: 0.9024
Epoch 5/5
88/88 [==============================] - 20s 223ms/step - loss: 0.1132 - accuracy: 0.9643 - val_loss: 0.2666 - val_accuracy: 0.9040
